In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info\PKG-INFO
writing dependency_links to mask_rcnn.egg-info\dependency_links.txt
writing top-level names to mask_rcnn.egg-info\top_level.txt
reading manifest file 'mask_rcnn.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\config.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\model.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\parallel_model.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\utils.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\visualize.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\__init__.py -> build\bdist.win-amd64\egg\mrcnn
byte-compiling build\bdist.win-amd64\egg\mrc

zip_safe flag not set; analyzing archive contents...



Installed d:\mask\mask\lib\site-packages\mask_rcnn-2.1-py3.7.egg
Processing dependencies for mask-rcnn==2.1
Finished processing dependencies for mask-rcnn==2.1


In [3]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from imgaug import augmenters as iaa
import scipy.misc
import matplotlib

In [4]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN


# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 9
    
# define 81 classes that the coco model knowns about
class_names = ['BG', 'Fire_truck','Ambulance','hiace','bus','truck','car','jeep','bike']


model = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())





Using TensorFlow backend.







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [5]:
model.load_weights('D:\\projects\\automatic _traffic_system\\models_for_eight_class\\mask_rcnn_eight_class_cfgh_0016.h5', by_name=True)

In [6]:
import cv2
import numpy as np


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
         print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [12]:
#for two lane 




def list_contains(List1, List2): 
  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        return True 
    else: 
        return False

import cv2 as cv
cap = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\ambulance100.mp4')
cap1 = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\woe2.mp4')
cap2 = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\1.mp4')
cap3 = cv.VideoCapture('D:\\projects\\automatic _traffic_system\\picture\\5.mp4')

size = (
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)


codec = cv2.VideoWriter_fourcc(*'DIVX')

#output1 = cv2.VideoWriter('D:\projects\automatic _traffic_system\tomorrow\\1lane1.avi', codec, 1.0, size)
#output2 = cv2.VideoWriter('D:\projects\automatic _traffic_system\tomorrow\\1lane2.avi', codec, 1.0, size)



if (cap.isOpened()== False):
    print("Error camera 1 isn't connecting")
if (cap1.isOpened()== False):
    print("Error camera 2 isn't connecting")
currentframe1=0

while (cap.isOpened() or cap1.isOpened()):
    ret, img = cap.read()
    ret1, img1 = cap1.read()
    ret2,img2=cap2.read()
    
    if ret == True:
        currentframe1 +=1
        #print(currentframe1)
        if ((currentframe1%30)==0):
            
                #for lane 1
                results1 = model.detect([img], verbose=0)
                r1 = results1[0]
                img = display_instances(
                img, r1['rois'], r1['masks'], r1['class_ids'], class_names, r1['scores'])
                #output1.write(img)
                cv2.imshow('lane1',img )
                print('lane1',r1['class_ids'])
                class_ids1 = r1['class_ids']
                emergency=[1,2]
                if list_contains(class_ids1, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 1')
                    print('red light on for lane 2 ')
                    print('red light on for lane 3')
                    print('red light on for lane 4')
                    #for yellow light after certain second
                    
                    continue
                
                else:
                    print('Number of vehicle in lane 1:',len(class_ids1))
                    class_ids1=list(class_ids1)
                    number_of_bikes_1=class_ids1.count(9)
                    if number_of_bikes_1>3:
                        number_of_car_equivalent=int(number_of_bikes_1/4)
                        class_ids1.extend([2]*number_of_car_equivalent)
                    else:
                        class_ids1 = [x for x in class_ids1 if x != 9]
                    density_of_lane1=len(class_ids1)
                    
                    
               

                #for lane 2
                results2 = model.detect([img1], verbose=0)
                r2 = results2[0]
                img1 = display_instances(
                img1, r2['rois'], r2['masks'], r2['class_ids'], class_names, r2['scores'])
                #output2.write(img1)
                cv2.imshow('lane2', img1)
                print('lane2',r2['class_ids'])
                class_ids2 = r2['class_ids']
                emergency=[1,2]
                if list_contains(class_ids2, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 2')
                    print('red light on for lane 1 ')
                    print('red light on for lane 3')
                    print('red light on for lane 4')
                    #
                    continue
                    
                else:
                    print('Number of vehicle in lane 2:',len(class_ids2))
                    class_ids2=list(class_ids2)
                    number_of_bikes_2=class_ids2.count(9)
                    if number_of_bikes_2>3:
                        number_of_car_equivalent=int(number_of_bikes_2/4)
                        class_ids2.extend([2]*number_of_car_equivalent)
                    else:
                        class_ids2 = [x for x in class_ids2 if x != 9]
                    density_of_lane2=len(class_ids2)
                    
                
                
                #for lane 3
                results3 = model.detect([img2], verbose=0)
                r3 = results3[0]
                img2 = display_instances(
                img2, r3['rois'], r3['masks'], r3['class_ids'], class_names, r3['scores'])
                #output2.write(img1)
                cv2.imshow('lane3', img2)
                print('lane2',r3['class_ids'])
                class_ids3 = r3['class_ids']
                emergency=[1,2]
                if list_contains(class_ids3, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 3')
                    continue
                    
                else:
                    print('Number of vehicle in lane 3:',len(class_ids3))
                    class_ids3=list(class_ids3)
                    number_of_bikes_3=class_ids3.count(9)
                    if number_of_bikes_3>3:
                        number_of_car_equivalent=int(number_of_bikes_3/4)
                        class_ids3.extend([2]*number_of_car_equivalent)
                    else:
                        class_ids3 = [x for x in class_ids3 if x != 9]
                    density_of_lane3=len(class_ids3)
                    
                
                
                
                
                
                
                
                
                
                
                results4 = model.detect([img3], verbose=0)
                r4 = results4[0]
                img3 = display_instances(
                img3, r3['rois'], r3['masks'], r3['class_ids'], class_names, r3['scores'])
                #output2.write(img1)
                cv2.imshow('lane4', img3)
                print('lane4',r4['class_ids'])
                class_ids4 = r4['class_ids']
                emergency=[1,2]
                if list_contains(class_ids4, emergency):
                    print('Emergency vehicle detected')
                    print('Green light is on for lane 4')
                    continue
                    
                    
                    
                else:
                    print('Number of vehicle in lane 4:',len(class_ids4))
                    class_ids4=list(class_ids4)
                    number_of_bikes_4=class_ids4.count(9)
                    if number_of_bikes_4>3:
                        number_of_car_equivalent=int(number_of_bikes_4/4)
                        class_ids4.extend([2]*number_of_car_equivalent)
                    else:
                        class_ids4 = [x for x in class_ids4 if x != 9]
                    density_of_lane4=len(class_ids4)
                    
                        
                        
                density_list=[density_of_lane1,density_of_lane2,density_of_lane3,density_of_lane4]        
                if ((max(density_list)==density_of_lane1) and  ):
                    print('green light on for lane 1')
                    print('red light on for lane 2 ')
                    print('red light on for lane 3')
                    print('red light on for lane 4')
                
                
                if (max(density_list)==density_of_lane2):
                    print('green light on for lane 2')
                    print('red light on for lane 1 ')
                    print('red light on for lane 3')
                    print('red light on for lane 4')
                    
                
                if (max(density_list)==density_of_lane3):
                    print('green light on for lane 3')
                    print('red light on for lane 1 ')
                    print('red light on for lane 2')
                    print('red light on for lane 4')
                    
                
                
                else:
                    print('green light on for lane 4')
                    print('red light on for lane 1 ')
                    print('red light on for lane 2')
                    print('red light on for lane 3')
                        
                




    if  cv.waitKey(1) & 0xFF == ord('q'):
        break
   


cap.release()
cap1.release()
cv.destroyAllWindows()

lane1 [8 6 2]
Emergency vehicle detected
Green light is on for lane 1
red light on for lane 2 
red light on for lane 3
red light on for lane 4
lane1 [2]
Emergency vehicle detected
Green light is on for lane 1
red light on for lane 2 
red light on for lane 3
red light on for lane 4
lane1 [8 2]
Emergency vehicle detected
Green light is on for lane 1
red light on for lane 2 
red light on for lane 3
red light on for lane 4
lane1 [2 6 8]
Emergency vehicle detected
Green light is on for lane 1
red light on for lane 2 
red light on for lane 3
red light on for lane 4


KeyboardInterrupt: 

In [ ]:
for i in b:
    if i==4:
        index=b.index(i)
        b.
    
    else:
        print('no')
    

In [ ]:
d=[1,2,3,5]

In [ ]:
d.index(5)

In [ ]:
d.remove(0)

In [ ]:
a = [10, 20, 30, 40, 20, 30, 40, 20, 70, 20]
a = [x for x in a if x != 20]

In [ ]:
e=[1,2,3,9,9]
e = [x for x in e if x != 9]
e